<center><img src='https://drive.google.com/uc?export=view&id=1qJ8NqAZolTBQY7lN-deZ8xEsU3dlUiLz' width=200></center>


<h6><center></center></h6>

<h1>
<hr style=" border:none; height:3px;">
<center>Cours XAI</center>
    <center> Activités 2 : Explications post-hoc sur données tabulaires</center>
<hr style=" border:none; height:3px;">
</h1>

# 1.Prérequis
N'oubliez pas d'installer avec pip (ou autre): xgboost, lime, shap

# 2.Mise en place d'un modèle boîte noire
Nous allons utiliser XGBoost pour entraîner un classifieur.

Dans un premier temps, chargez le fichier de données 'insurance_propre.csv', qui correspond au même jeu de données que la dernière fois mais déjà pré-traité.

In [ ]:
from pandas import *
data = read_csv(r'insurance_propre.csv', sep=';')

data.info()

A présent, nous allons découper ce dataset en un ensemble d'apprentissage (80%) et un autre de test (20%).

In [ ]:
from sklearn.model_selection import train_test_split

X = data.drop('charges', axis=1)
y = data['charges']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

A présent, nous allons entraîner XGBoost avec ces données d'apprentissage. Pour cela, nous allons utiliser [XGBRegressor](https://xgboost.readthedocs.io/en/latest/python/python_api.html).

In [ ]:
pip install XGBoost

In [ ]:
import xgboost as xgb

regressor = xgb.XGBRegressor(
    n_estimators=100,
    reg_lambda=1,
    gamma=0,
    max_depth=3
)
regressor.fit(X_train.values, y_train.values)

Il est possible d'inspecter le modèle et de regarder l'importance des features.

In [ ]:
DataFrame(regressor.feature_importances_.reshape(1, -1), columns=X_train.columns)

Est-ce que cela vous semble cohérent ?

In [ ]:
#réponse

## LIME
Dans cette partie, nous allons appliquer LIME au modèle ci-dessus. Dans un premier temps, au lieu de générer de données, nous allons entraîner LIME sur les données d'apprentissage.
Pour cela, regardez la classe [LimeTabularExplainer](https://lime-ml.readthedocs.io/en/latest/lime.html#lime.lime_tabular.LimeTabularExplainer).

In [ ]:
import lime
from lime import lime_tabular
import numpy as np

explainer = lime_tabular.LimeTabularExplainer(
    training_data=np.array(X_train),
    feature_names=X_train.columns,
    class_names=['charges'],
    mode='regression'
)

A présent, nous allons pouvoir étudier le premier élément de l'ensemble de test.

In [ ]:
print(X_test.columns)

exp = explainer.explain_instance(
    data_row=X_test.iloc[1], 
    predict_fn=regressor.predict,
    num_features = 9
)

exp.show_in_notebook(show_table=True)

#Interprétez le résultat

## SHAP
A présent, nous allons nous intéresser à [SHAP](https://shap.readthedocs.io/en/latest/index.html), une autre alternative pour mieux comprendre les décisions de notre modèle.
Comme nous l'avons vu, SHAP est complexe en termes de calculs. Nous devons réduire l'ensemble d'apprentissage pour pouvoir l'utiliser dans un temps raisonnable.

In [ ]:
import shap
train = shap.sample(X_train,10)

Créez l'explainer à partir de cet ensemble train. Utilisez KernelExplainer car XGBoost n'est pas supporté pat TreeExplainer.

In [ ]:
explainer = shap.KernelExplainer(regressor.predict, train)

A présent, récupérer les valeurs de shapley avec la méthode shap_values sur les exemples de train et affichez le bilan avec summary_plot.

In [ ]:
shap_values = explainer.shap_values(train)
print(shap_values)
shap.summary_plot(shap_values, train)

SHAP propose également de nombreuses visualisations, comme waterfall. Il faut créer un [Explainer général](https://shap.readthedocs.io/en/latest/generated/shap.Explainer.html). Cette visualisation nous donne la participation des features à la décision.

In [ ]:
explainer = shap.Explainer(regressor)
shap_values = explainer(X_test)

# On prend le 2e exemple
shap.plots.waterfall(shap_values[1])

Que concluez-vous de l'analyse SHAP ?

In [ ]:
#réponse